In [1]:
import os
import numpy as np
import pandas as pd
import re
import math
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_rows', None)

In [2]:
url1 = "https://m.bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_base_stats_(Generation_VIII-present)"

In [3]:
html = requests.get(url1).content
soup = BeautifulSoup(html, "lxml")

In [4]:
table = soup.find_all("table", {"class":"sortable roundy"})[0]

In [5]:
rows = table.find_all("tr")
rows = [row.text.strip().split("\n") for row in rows]
rows = [list(filter(None,row)) for row in rows]
bulbapedia_df = pd.DataFrame(rows)
bulbapedia_df.columns = bulbapedia_df.iloc[0]
bulbapedia_df = bulbapedia_df.drop([0])
bulbapedia_df.columns = ["#N.Dex", "pic", "Pokémon", "HP", "Attack", "Defense",
                         "Sp. Attack", "Sp. Defense", "Speed", "Total", "Average"]
bulbapedia_df_clean=bulbapedia_df.drop(["pic"], axis=1)

In [6]:
bulbapedia_df_clean.head()

,#N.Dex,Pokémon,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Total,Average
1,001,Bulbasaur,45,49,49,65,65,45,318,53
2,002,Ivysaur,60,62,63,80,80,60,405,67.5
3,003,Venusaur,80,82,83,100,100,80,525,87.5
4,003,Venusaur (Mega Venusaur),80,100,123,122,120,80,625,104.17
5,004,Charmander,39,52,43,60,50,65,309,51.5


In [7]:
bulbapedia_df_clean.dtypes

#N.Dex         object
Pokémon        object
HP             object
Attack         object
Defense        object
Sp. Attack     object
Sp. Defense    object
Speed          object
Total          object
Average        object
dtype: object

In [8]:
columns=["HP", "Attack", "Defense","Sp. Attack",
         "Sp. Defense", "Speed", "Total", "Average"]
for column in columns:
    bulbapedia_df_clean[column]= bulbapedia_df_clean[column].astype(float)

In [9]:
print("Are there any missing values? :",bulbapedia_df_clean.isnull().any())

Are there any missing values? : #N.Dex         False
Pokémon        False
HP             False
Attack         False
Defense        False
Sp. Attack     False
Sp. Defense    False
Speed          False
Total          False
Average        False
dtype: bool


In [10]:
print("Are there duplicated values? :",bulbapedia_df_clean.duplicated().any())

Are there duplicated values? : False


In [11]:
bool_series = bulbapedia_df_clean["Pokémon"].str.endswith(")") 
  
bulbapedia_df_clean[bool_series] 

,#N.Dex,Pokémon,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Total,Average
4,003,Venusaur (Mega Venusaur),80.0,100.0,123.0,122.0,120.0,80.0,625.0,104.17
8,006,Charizard (Mega Charizard X),78.0,130.0,111.0,130.0,85.0,100.0,634.0,105.67
9,006,Charizard (Mega Charizard Y),78.0,104.0,78.0,159.0,115.0,100.0,634.0,105.67
13,009,Blastoise (Mega Blastoise),79.0,103.0,120.0,135.0,115.0,78.0,630.0,105.00
20,015,Beedrill (Mega Beedrill),65.0,150.0,40.0,15.0,80.0,145.0,495.0,82.50
24,018,Pidgeot (Mega Pidgeot),83.0,80.0,80.0,135.0,80.0,121.0,579.0,96.50
26,019,Rattata (Alolan Rattata),30.0,56.0,35.0,25.0,35.0,72.0,253.0,42.17
28,020,Raticate (Alolan Raticate),75.0,71.0,70.0,40.0,80.0,77.0,413.0,68.83
34,025,Pikachu (Partner Pikachu),45.0,80.0,50.0,75.0,60.0,120.0,430.0,71.67
36,026,Raichu (Alolan Raichu),60.0,85.0,50.0,95.0,85.0,110.0,485.0,80.83


In [12]:
for row in bulbapedia_df_clean["Pokémon"]:
    bulbapedia_df_clean["Pokémon"]= bulbapedia_df_clean["Pokémon"].replace("Landorus (Therian Forme)","Landorus").astype(str)
    bulbapedia_df_clean["Pokémon"]= bulbapedia_df_clean["Pokémon"].replace("Thundurus (Therian Forme)","Thundurus").astype(str)
    bulbapedia_df_clean["Pokémon"]= bulbapedia_df_clean["Pokémon"].replace("Aegislash (Shield Forme)","Aegislash").astype(str)
    bulbapedia_df_clean["Pokémon"]= bulbapedia_df_clean["Pokémon"].replace("Thundurus (Therian Forme)","Thundurus").astype(str)
    bulbapedia_df_clean["Pokémon"]= bulbapedia_df_clean["Pokémon"].replace("Kyurem (Normal Kyurem)","Kyurem").astype(str)

In [13]:
bulbapedia_df_clean.to_csv("./data/bulbapedia_clean.csv",index=False)